# Connection

> Helps Manage Snowflake Connection

In [3]:
#| default_exp procedures

In [4]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from pathlib import Path
from datetime import datetime
from snowflake.snowpark import Session

from typing import Optional, Dict, Union
import sys
import json
import os
import yaml
import logging
import zipfile

In [6]:
#| export


def is_snowflake_environment() -> bool:
    """Detect if code is running in Snowflake environment."""
    return hasattr(sys, '_xoptions') and 'snowflake_import_directory' in sys._xoptions

In [7]:
is_snowflake_environment()

False

In [18]:
#| export


def find_config_file(filename: str = 'config.yaml', max_levels_up: int = 3) -> Optional[Path]:
    """
    Search for config file in current directory and parent directories.
    
    Args:
        filename: Name of config file to find
        max_levels_up: Maximum number of parent directories to check
    
    Returns:
        Path to config file if found, None otherwise
    """
    current = Path.cwd()
    
    # First check if config exists in expected structure from current directory
    expected_path = current / 'customfunctions' / 'files' / filename
    if expected_path.is_file():
        return expected_path
        
    # Then check parent directories
    for _ in range(max_levels_up):
        # Check for config in customfunctions/files structure
        config_path = current / 'customfunctions' / 'files' / filename
        if config_path.is_file():
            return config_path
            
        # Also check for direct config.yaml in case structure is different
        direct_path = current / filename
        if direct_path.is_file():
            return direct_path
            
        current = current.parent
        
    return None

def get_config_path(snowflake_extracted_path: str = '/tmp/customfunctions') -> Path:
    """
    Determine the configuration file path based on environment.
    
    Args:
        snowflake_extracted_path: Path where Snowflake extracts files
    
    Returns:
        Path object pointing to config file
    """
    if is_snowflake_environment():
        # Snowflake environment logic remains the same
        import_dir = Path(sys._xoptions['snowflake_import_directory'])
        zip_file_path = import_dir / "customfunctions.zip"
        
        if not Path(snowflake_extracted_path).exists():
            try:
                with zipfile.ZipFile(zip_file_path, 'r') as myzip:
                    myzip.extractall(snowflake_extracted_path)
            except Exception as e:
                logging.error(f"Failed to extract ZIP file: {e}")
                raise
                
        return Path(snowflake_extracted_path) / 'customfunctions/files/config.yaml'
    else:
        # Local environment - search for config file
        config_path = find_config_file()
        if config_path is None:
            # If not found, return a default path for consistent error messaging
            return Path.cwd() / 'customfunctions' / 'files' / 'config.yaml'
        return config_path

In [19]:
get_config_path()

Path('/Users/jdemlow/github/customfunctions/customfunctions/files/config.yaml')

In [11]:
#| export


def load_config(config_path: Path) -> Dict:
    """
    Load configuration from YAML file with error handling.
    
    Args:
        config_path: Path to configuration file
    
    Returns:
        Dictionary containing configuration
    """
    try:
        if config_path.is_file():
            with open(config_path, 'r') as file:
                config = yaml.safe_load(file)
                logging.info(f"Successfully loaded configuration from {config_path}")
                return config or {}
        else:
            logging.warning(f"Configuration file '{config_path}' not found. Using empty configuration.")
            return {}
    except Exception as e:
        logging.error(f"Error reading configuration file: {e}")
        return {}

In [22]:
load_config(get_config_path())

{'added_word': 'YAML FOR THE WIN'}

In [23]:
load_config(Path('../customfunctions/files/config.yaml'))

{'added_word': 'YAML FOR THE WIN'}

In [24]:
#| export


def hello_procedure(session: Optional[Session] = None, name: str = "World") -> str:
    """
    Enhanced hello procedure that works both locally and in Snowflake.
    
    Args:
        session: Snowflake session (optional, only needed in Snowflake environment)
        name: Name to greet
    
    Returns:
        Greeting string
    """
    try:
        # Get config path based on environment
        config_path = get_config_path()
        
        # Load configuration
        config = load_config(config_path)
        
        # Get additional word from config, with fallback
        added_word = config.get('added_word', '')
        
        # Build and return greeting
        greeting = f"Hello, {name}!"
        if added_word:
            greeting += f" {added_word}"
            
        return greeting
        
    except Exception as e:
        logging.error(f"Error in hello_procedure: {e}")
        # Return a basic greeting in case of any errors
        return f"Hello, {name}!"


In [26]:
# Set up logging
logging.basicConfig(level=logging.INFO)

# Test local execution
result = hello_procedure(session=Session, name="Local User")
print(f"Local test result: {result}")

Local test result: Hello, Local User! YAML FOR THE WIN


In [27]:
# | export
def test_procedure(session: Session) -> str:
    return "Test procedure"

In [29]:
#| hide
import nbdev; nbdev.nbdev_export()